In [45]:
import pandas as pd
import requests
# Import the required libraries and dependencies
import questionary
import import_ipynb
import os
import fire
import argparse
import click
import time
import requests
import json
import locale
import urllib.parse
import hashlib
import hmac
import base64
#import app
import krakenex
from time import sleep
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)

from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

from dotenv import load_dotenv
# from MCForecastTools import MCSimulation


%matplotlib inline

In [46]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [47]:
api_key = os.getenv("API_KEY_KRAKEN")
api_sec = os.getenv("API_SEC_KRAKEN")
api_url = "https://api.kraken.com"


In [48]:
# Verify the Key and Secret Key
print(os.environ.get('API_KEY_KRAKEN'))
print(os.environ.get('API_SEC_KRAKEN'))
type(api_key)
type(api_sec)


51b6FJZPTY1qzkgS6SzK18XZyclYB/FVQ18V0AfvateucQ9a1AADHW1h
p1xQNrAoynfMs0gRvFV0NO3pitwdmrUU6G/4KCdt6FtB0Q4B2klxzvJkMAEuGw+CjQs3jfkQdCwsmPSGlLxTtQ==


str

In [49]:
def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

In [50]:
# Attaches auth headers and returns results of a POST request
def kraken_request(uri_path, data, api_key, api_sec):
    headers = {}
    headers['API-Key'] = api_key
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)             
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    print(api_url + uri_path)
    print(headers['API-Key'])
    print(headers['API-Sign'])
    return req

In [51]:
supported_currencies = requests.get('https://api.coingecko.com/api/v3/simple/supported_vs_currencies').json()
#Asset_Info
#print(json.dumps(supported_currencies, indent=4, sort_keys=True))



In [52]:
ticker = k.get_ticker_information('BTCUSD')
ticker.T

,XXBTZUSD
a,"[30689.10000, 1, 1.000]"
b,"[30687.80000, 1, 1.000]"
c,"[30690.30000, 0.28027290]"
h,"[30900.00000, 30900.00000]"
l,"[29281.80000, 29062.10000]"
o,29450.50000
p,"[30418.97709, 30307.22909]"
t,"[20037, 24056]"
v,"[3298.53467615, 3660.15061868]"


In [53]:
cg.get_price(ids='bitcoin', vs_currencies='usd')

{'bitcoin': {'usd': 30719}}

In [54]:
cg.get_price(ids=['bitcoin', 'ethereum','litecoin'], vs_currencies='usd')

{'bitcoin': {'usd': 30719},
 'ethereum': {'usd': 1927.65},
 'litecoin': {'usd': 66.94}}

In [67]:
Coins_list = cg.get_coins_list()[0:10]
print(json.dumps(Coins_list, indent=4, sort_keys=True))

type(k)

[
    {
        "id": "01coin",
        "name": "01coin",
        "symbol": "zoc"
    },
    {
        "id": "0-5x-long-algorand-token",
        "name": "0.5X Long Algorand",
        "symbol": "algohalf"
    },
    {
        "id": "0-5x-long-altcoin-index-token",
        "name": "0.5X Long Altcoin Index",
        "symbol": "althalf"
    },
    {
        "id": "0-5x-long-ascendex-token-token",
        "name": "0.5X Long AscendEx",
        "symbol": "asdhalf"
    },
    {
        "id": "0-5x-long-bitcoin-cash-token",
        "name": "0.5X Long Bitcoin Cash",
        "symbol": "bchhalf"
    },
    {
        "id": "0-5x-long-bitcoin-token",
        "name": "0.5X Long Bitcoin",
        "symbol": "half"
    },
    {
        "id": "0-5x-long-cardano-token",
        "name": "0.5X Long Cardano",
        "symbol": "adahalf"
    },
    {
        "id": "0-5x-long-chainlink-token",
        "name": "0.5X Long Chainlink",
        "symbol": "linkhalf"
    },
    {
        "id": "0-5x-long-cosmos-token

pykrakenapi.pykrakenapi.KrakenAPI

In [56]:
data = {
    "nonce": "1616492376594", 
    "ordertype": "limit", 
    "pair": "XBTUSD",
    "price": 37500, 
    "type": "buy",
    "volume": 1.25
}

signature = get_kraken_signature("/0/private/AddOrder", data, api_sec)
print("API-Sign: {}".format(signature))

API-Sign: Z0qFs0BeSK/35eX+y6Ez2/eZU/dZrcMYCYOmBwR2diikw7/nM7H6x/dNaVyQoH1mNf3z0unmBs0opk0AsUjWRg==


In [57]:
Get_Kraken_time = requests.get('https://api.kraken.com/0/public/Time')
Kraken_time = Get_Kraken_time.json()
print(json.dumps(Kraken_time, indent=4, sort_keys=True))



{
    "error": [],
    "result": {
        "rfc1123": "Mon, 30 May 22 18:12:50 +0000",
        "unixtime": 1653934370
    }
}


In [58]:
Get_Online_Offline = requests.get('https://api.kraken.com/0/public/SystemStatus')
Online_Offline = Get_Online_Offline .json()
print(json.dumps(Online_Offline, indent=4, sort_keys=True))

{
    "error": [],
    "result": {
        "status": "online",
        "timestamp": "2022-05-30T18:12:51Z"
    }
}


In [59]:
# Construct the request and print the result
resp1 = kraken_request('/0/private/AddOrder', {
    "nonce": str(int(1000*time.time())),
    "ordertype": "limit",
    "type": "buy",
    "volume": 1.25,
    "pair": "XBTUSD",
    "price": 27500
}, api_key, api_sec)



print(resp1.json())

https://api.kraken.com/0/private/AddOrder
51b6FJZPTY1qzkgS6SzK18XZyclYB/FVQ18V0AfvateucQ9a1AADHW1h
74nsU/gGVY3hRMH0d9s83hfFFPbBiuiet4tWEljzzLP13Hc7mZzlnqo9e01+kO7GnxKVJ50ZUhUal6lU00vR2w==
{'error': ['EGeneral:Internal error']}


In [60]:
## Get Trade Balance
# Construct the request and print the result

Get_trade_balances = kraken_request('/0/private/TradeBalance', {
    "nonce": str(int(1000*time.time())),
    "asset": "USD"
}, api_key, api_sec)

Get_trade_balances = Get_trade_balances.json()
print(json.dumps(Get_trade_balances, indent=4, sort_keys=True))

https://api.kraken.com/0/private/TradeBalance
51b6FJZPTY1qzkgS6SzK18XZyclYB/FVQ18V0AfvateucQ9a1AADHW1h
hk+LR+MoEf+nv/fCre1XYrCme6IuwTcYu23XdlC8IZWk58/7h5iBSrsnpNCy3/n19S3zeEnHC3/PZeCy3f638Q==
{
    "error": [],
    "result": {
        "c": "0.0000",
        "e": "0.0000",
        "eb": "0.0000",
        "m": "0.0000",
        "mf": "0.0000",
        "n": "0.0000",
        "tb": "0.0000",
        "v": "0.0000"
    }
}


In [61]:
# Attaches auth headers and returns results of a POST request

# Construct the request and print the result
Kraken_Add_Order = kraken_request('/0/private/AddOrder', {
    "nonce": str(int(1000*time.time())),
    "ordertype": "limit",
    "type": "buy",
    "volume": 1.25,
    "pair": "XBTUSD",
    "price": 27500
}, api_key, api_sec)


display(data)
display(api_key)
display(api_sec)

Kraken_Add_Orders  = Kraken_Add_Order.json()
print(json.dumps(Kraken_Add_Orders, indent=4, sort_keys=True))

https://api.kraken.com/0/private/AddOrder
51b6FJZPTY1qzkgS6SzK18XZyclYB/FVQ18V0AfvateucQ9a1AADHW1h
oI7hgUNYNC5ryAw5VeZhNkCLjP8scmdVI/4+9PJDkaFGnKrEAW1HZdJxuzQqJWTUjRkagnkdB1T5OMviqUyxvg==


{'nonce': '1616492376594',
 'ordertype': 'limit',
 'pair': 'XBTUSD',
 'price': 37500,
 'type': 'buy',
 'volume': 1.25}

'51b6FJZPTY1qzkgS6SzK18XZyclYB/FVQ18V0AfvateucQ9a1AADHW1h'

'p1xQNrAoynfMs0gRvFV0NO3pitwdmrUU6G/4KCdt6FtB0Q4B2klxzvJkMAEuGw+CjQs3jfkQdCwsmPSGlLxTtQ=='

{
    "error": [
        "EGeneral:Internal error"
    ]
}


In [62]:
# Construct the request and print the result
AddOrder = kraken_request('/0/private/AddOrder', {
    "nonce": str(int(1000*time.time())),
    "ordertype": "limit",
    "type": "buy",
    "volume": 1.25,
    "pair": "XBTUSD",
    "price": 27500
}, api_key, api_sec)

print(AddOrder.json())

https://api.kraken.com/0/private/AddOrder
51b6FJZPTY1qzkgS6SzK18XZyclYB/FVQ18V0AfvateucQ9a1AADHW1h
dZ5S9DxnRY1CEDbMmJvGYrK7UQBz0ZqZiulXBXiAOwAQcT0Siq1MVEuY3zUQ0P/8pjLbya5GTidRPhG4xNTZ9w==
{'error': ['EGeneral:Internal error']}


In [63]:
## Get Account Balance

# Construct the request and print the result
Get_Account_Balance = kraken_request('/0/private/Balance', {
    "nonce": str(int(1000*time.time()))
}, api_key, api_sec)

Account_Balance = Get_Account_Balance.json()
print(json.dumps(Account_Balance, indent=4, sort_keys=True)) 


https://api.kraken.com/0/private/Balance
51b6FJZPTY1qzkgS6SzK18XZyclYB/FVQ18V0AfvateucQ9a1AADHW1h
TbSoA1ccNHF/ZMesa1ADN00QWVBacWHfDuJK3fPSvwR5gAuqlVBary8c14tYJuq+Nita4UTwBSd3qlBo6jAVSw==
{
    "error": [],
    "result": {}
}


In [64]:
# The Free Crypto API Call endpoint URLs for the held cryptocurrency assets
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

In [65]:
# Using the Python requests library, make an API call to access the current price of BTC
btc_response = requests.get(btc_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(btc_response, indent=4, sort_keys=True))

{
    "data": {
        "1": {
            "circulating_supply": 19053787,
            "id": 1,
            "last_updated": 1653934167,
            "max_supply": 21000000,
            "name": "Bitcoin",
            "quotes": {
                "USD": {
                    "market_cap": 585715496587,
                    "percent_change_1h": 0.189174608634593,
                    "percent_change_24h": 5.00888930117943,
                    "percent_change_7d": 1.1955078855983,
                    "percentage_change_1h": 0.189174608634593,
                    "percentage_change_24h": 5.00888930117943,
                    "percentage_change_7d": 1.1955078855983,
                    "price": 30714.0,
                    "volume_24h": 26796699551
                }
            },
            "rank": 1,
            "symbol": "BTC",
            "total_supply": 19053787,
            "website_slug": "bitcoin"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrencies": 

In [66]:
# Using the Python requests library, make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Use the json.dumps function to review the response data from the API call
# Use the indent and sort_keys parameters to make the response object readable
print(json.dumps(eth_response, indent=4, sort_keys=True))

{
    "data": {
        "1027": {
            "circulating_supply": 120984849,
            "id": 1027,
            "last_updated": 1653934174,
            "max_supply": 0,
            "name": "Ethereum",
            "quotes": {
                "USD": {
                    "market_cap": 233339456960,
                    "percent_change_1h": 0.5356377385171,
                    "percent_change_24h": 7.00058057105278,
                    "percent_change_7d": -5.79975540998392,
                    "percentage_change_1h": 0.5356377385171,
                    "percentage_change_24h": 7.00058057105278,
                    "percentage_change_7d": -5.79975540998392,
                    "price": 1927.95,
                    "volume_24h": 15370246980
                }
            },
            "rank": 2,
            "symbol": "ETH",
            "total_supply": 120984849,
            "website_slug": "ethereum"
        }
    },
    "metadata": {
        "error": null,
        "num_cryptocurrencies